Creating EcoInvent Matrix from .spold files<br>
See: https://v34.ecoquery.ecoinvent.org/File/Files
<br>
<br>
At bottom in supporting documents there are instructions for matrix building from spold<br>
First download and unzip relevant system files - you may need a .7z to zip file converter

In [5]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import scipy as sp
import pandas as pd
from scipy import sparse
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import inv
from numpy.linalg import inv

First navigate to the appropriate folder - should be something like ~/Documents/D/Ecoinvent/ecoinvent-3.4_cutoff_ecoSpold02/

In [6]:
os.chdir("datasets")
dir = os.getcwd()

Change file extensions from .spold to .xml

In [7]:
for f in os.listdir():
    if f.endswith('.spold'):
        base = os.path.splitext(f)[0]
        target = os.path.join(dir, f)
        f = os.rename(target, base + '.xml')

Parse each file using ElementTree, extract key attributes as described in step 1 in the Ecoinvent documentation and put them in a dataframe. This is the index for the technosphere exchanges.

In [ ]:
colnames = ['act_id', 'ref_id', 'act_name', 'ref_name', 'geog']
idx = pd.DataFrame(data=[], index=list(range(0,len(os.listdir()))), columns=colnames)
for i in range(0, len(os.listdir())):
    tree = ET.parse(os.listdir()[i])
    root = tree.getroot()
    for activity in root.iter('{http://www.EcoInvent.org/EcoSpold02}activity'):
        idx.loc[i]['act_id'] = activity.attrib['id']
    for activityName in root.iter('{http://www.EcoInvent.org/EcoSpold02}activityName'):
        idx.loc[i]['act_name'] = activityName.text   
    for geog in root.findall('.//{http://www.EcoInvent.org/EcoSpold02}geography/{http://www.EcoInvent.org/EcoSpold02}shortname'):
        idx.loc[i]['geog'] = geog.text
    for IE in root.findall(".//{http://www.EcoInvent.org/EcoSpold02}intermediateExchange/[{http://www.EcoInvent.org/EcoSpold02}outputGroup='0']"):
        idx.loc[i]['ref_id'] = IE.attrib['intermediateExchangeId']
    for rf in root.findall(".//{http://www.EcoInvent.org/EcoSpold02}intermediateExchange/[{http://www.EcoInvent.org/EcoSpold02}outputGroup='0']/{http://www.EcoInvent.org/EcoSpold02}name"):
        idx.loc[i]['ref_name'] = rf.text
    del tree
    del root

Check to make sure it looks right:

In [136]:
idx.head(5)

,act_id,ref_id,act_name,ref_name,geog
0,c276df5a-9f59-4304-a25d-5d33d07303e9,1f41586d-0d8a-4c7c-8473-dd8351bab538,clinker production,clinker,US
1,679932c4-15f4-46d0-82e0-e126ee6ccb05,d69294d7-8d64-4915-a896-9996a014c410,electricity voltage transformation from medium...,"electricity, low voltage",SA
2,4c99ba2a-d1e2-4c41-910d-6ce77382db63,91667712-8403-49bd-95be-c929b47067c2,"market for transport, freight, sea, transocean...","transport, freight, sea, transoceanic ship",GLO
3,8fdf3ecb-6c94-4731-bca6-3cc2a5d6796b,0dab73c6-b214-4e9c-8c38-ab49d608637b,market for protein pea,protein pea,GLO
4,7fc96034-d15c-4ac1-b183-89c346a72821,cbcd0e88-5e08-4e2a-8353-ae6a4ba53ce2,"market for photovoltaic mounting system, for 5...","photovoltaic mounting system, for 570kWp open ...",GLO


Now navigate to Elementary Exchanges file to build an index for the elemental exchanges. This time there is only one file.

In [141]:
os.chdir("..")
os.chdir("MasterData")

In [247]:
tree = ET.parse('ElementaryExchanges.xml')
root = tree.getroot()
colnames = ['elex_id', 'subc_id', 'elex_name', 'comp_name', 'subc_name']
L = len(root.findall(".//{http://www.EcoInvent.org/EcoSpold02}elementaryExchange"))
elemidx = pd.DataFrame(data=[], index=list(range(0,L)), columns=colnames)

In [273]:
x = [None]*L
for i, elex in enumerate(root.iter('{http://www.EcoInvent.org/EcoSpold02}elementaryExchange')):
    x[i] = elex.attrib['id']
elemidx['elex_id'] = x
for i, subc in enumerate(root.findall('.//{http://www.EcoInvent.org/EcoSpold02}elementaryExchange/{http://www.EcoInvent.org/EcoSpold02}compartment')):
    x[i] = subc.attrib['subcompartmentId']
elemidx['subc_id'] = x
for i, lname in enumerate(root.iter('{http://www.EcoInvent.org/EcoSpold02}name')):
    x[i] = lname.text
elemidx['elex_name'] = x
for i, comp in enumerate(root.findall('.//{http://www.EcoInvent.org/EcoSpold02}elementaryExchange/{http://www.EcoInvent.org/EcoSpold02}compartment/{http://www.EcoInvent.org/EcoSpold02}compartment')):
    x[i] = comp.text
elemidx['comp_name'] = x
for i, sname in enumerate(root.iter('{http://www.EcoInvent.org/EcoSpold02}subcompartment')):
    x[i] = sname.text
elemidx['subc_name'] = x

Check to make sure it looks right:

In [296]:
elemidx.head(5)

,elex_id,subc_id,elex_name,comp_name,subc_name
0,38a622c6-f086-4763-a952-7c6b3b1c42ba,e8d7772c-55ca-4dd7-b605-fee5ae764578,"1,4-Butanediol",air,urban air close to ground
1,541a823c-0aad-4dc4-9123-d4af4647d942,e8d7772c-55ca-4dd7-b605-fee5ae764578,1-Pentanol,air,urban air close to ground
2,8cbaa905-41b0-4327-8403-bf1c8eb25429,e8d7772c-55ca-4dd7-b605-fee5ae764578,1-Pentene,air,urban air close to ground
3,f681eb3c-854a-4f78-bcfe-76dfbcf9df3c,e1bc9a16-5b6a-494f-98ef-49f461b1a11e,"2,4-D",soil,agricultural
4,a0fec60d-3f74-48bf-a2d2-58c30fc13e53,e8d7772c-55ca-4dd7-b605-fee5ae764578,2-Aminopropanol,air,urban air close to ground


We now have 2 index dataframes, elemidx for the elemental flows and idx for the technosphere inputs and outputs. Now we have to build the actual matrices: The A (technology) matrix, and the B (elementary flow) matrix. First create two matrices of zeros of the appropriate size.

In [4]:
A = np.zeros((len(idx),len(idx)))
B = np.zeros((len(elemidx),len(idx)))

NameError: name 'idx' is not defined

Now we need to parse each of the spold files to find the coefficients, and put them in the right place in the matrix based on their index numbers. First navigate back to the datasets folder.

In [298]:
os.chdir("..")
os.chdir("datasets")

Next, open each file, parse the tree, identify the correct column in the matrix, find the coefficients and put them in. Off diagonal coefficients in the A matrix are multiplied by -1 as described in the EcoInvent documentation.

In [ ]:
for i in range(0, len(os.listdir())):
    tree = ET.parse(os.listdir()[i])
    root = tree.getroot()
    for activity in root.iter('{http://www.EcoInvent.org/EcoSpold02}activity'):
        actid = activity.attrib['id']
    for IE in root.findall(".//{http://www.EcoInvent.org/EcoSpold02}intermediateExchange/[{http://www.EcoInvent.org/EcoSpold02}outputGroup='0']"):
        refid = IE.attrib['intermediateExchangeId']
        amt = float(IE.attrib['amount'])
    q = idx.index.values[(idx['act_id']==actid) & (idx['ref_id']==refid)]  ##this is the column index for A and B matrices
    q = q[0] 
    A[q][q] = amt  ## place the reference flow on the diagonal
    
    ## create some temporary empty lists
    k = []
    l = []  
    m = []
    n = []
    o = []
    p = []
    r = []
    s = []
    t = []
    u = []
    v = []
    z = []
    
    ## populate the A matrix
    for j, IE in enumerate(root.findall(".//{http://www.EcoInvent.org/EcoSpold02}intermediateExchange/[{http://www.EcoInvent.org/EcoSpold02}inputGroup='5']")):
            k.append(IE.attrib['intermediateExchangeId'])
            l.append(IE.attrib['activityLinkId'])
            w = idx.index.values[(idx['act_id']==l[j]) & (idx['ref_id']==k[j])] ## find index for the intermediate exchange
            m.append(w[0])
            n.append(-1*float(IE.attrib['amount'])) ## multiply coefficient by -1
    qc = A[:,q]
    np.put(qc, m, n)
    A[:,q] = qc
    
    ## populate the B matrix
    
    qb = B[:,q]
    np.put(qb, s, p)
    np.put(qb, u, z)
    B[:,q] = qb
    
    ## clear the temporary variables
    del k
    del l
    del m
    del n
    del o 
    del p
    del r
    del s
    del t
    del u
    del v
    del z
    del qc
    del qb
    del w
    del tree
    del root
    del actid
    del refid

In [494]:
for j, LX in enumerate(root.findall(".//{http://www.EcoInvent.org/EcoSpold02}elementaryExchange/[{http://www.EcoInvent.org/EcoSpold02}inputGroup='4']")):
        o.append(LX.attrib['elementaryExchangeId'])
        p.append(float(LX.attrib['amount']))
        for c in root.find(".//{http://www.EcoInvent.org/EcoSpold02}elementaryExchange[@elementaryExchangeId==o[j]].../{http://www.EcoInvent.org/EcoSpold02}compartment"):       
            r.append(c.attrib['subcompartmentId'])
            w = elemidx.index.values[(idx['elex_id']==o[j]) & (idx['subc_id']==r[j])]
            s.append(w[0])

SyntaxError: invalid predicate (<string>)

In [505]:
        for y, c in root.findall(".//{http://www.EcoInvent.org/EcoSpold02}elementaryExchange/{http://www.EcoInvent.org/EcoSpold02}compartment"):       
            r.append(c.attrib['subcompartmentId'])

{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{'{http://www.w3.org/XML/1998/namespace}lang':

In [502]:
del y

In [486]:
    for j, LX in enumerate(root.findall(".//{http://www.EcoInvent.org/EcoSpold02}elementaryExchange/[{http://www.EcoInvent.org/EcoSpold02}outputGroup='4']")):
            t.append(LX.attrib['elementaryExchangeId'])
            u.append(float(LX.attrib['amount']))    
            
            
            v.append(LX.attrib['subcompartmentId'])
            w = elemidx.index.values[(idx['elex_id']==t[j]) & (idx['subc_id']==v[j])]
            z.append(w[0])

In [ ]:
A_inv = np.linalg.inv(A)

In [552]:
A_sp = sp.sparse.csc_matrix(A)
A_inv = sp.sparse.linalg.inv(A_sp)

RuntimeError: Factor is exactly singular

In [541]:
f = np.zeros(len(A))

In [540]:
i = idx.index[(idx['ref_name']=='protein pea')]
idx.iloc[i]

,act_id,ref_id,act_name,ref_name,geog
3,8fdf3ecb-6c94-4731-bca6-3cc2a5d6796b,0dab73c6-b214-4e9c-8c38-ab49d608637b,market for protein pea,protein pea,GLO
6,a484a5dc-a40e-4129-9f3b-7209d9dc9c3d,0dab73c6-b214-4e9c-8c38-ab49d608637b,protein pea production,protein pea,RoW
2810,113193a6-67f8-48d5-b5c9-3e7480e5fe97,0dab73c6-b214-4e9c-8c38-ab49d608637b,protein pea production,protein pea,FR
7987,9988c640-2abc-4ff3-8262-1e22d27b3db8,0dab73c6-b214-4e9c-8c38-ab49d608637b,protein pea production,protein pea,ES
12016,1e0dcced-d91e-4d44-a94a-36c1a8963ad3,0dab73c6-b214-4e9c-8c38-ab49d608637b,protein pea production,protein pea,DE


In [545]:
np.put(f,[3,6],1)